In [ ]:
import pandas as pd
import numpy as np
import panel as pn
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Legend, LegendItem, FactorRange
from scipy.stats import pearsonr

# Initialize Panel extension
pn.extension()

# Define utility functions for calculations
def calculate_percent_bias(observed, predicted):
    """Calculate percent bias between observed and predicted values"""
    return 100 * np.sum(predicted - observed) / np.sum(observed)

def calculate_metrics(observed, predicted):
    """Calculate correlation squared, MAE, and percent bias"""
    # Remove NaN values from both arrays simultaneously
    mask = ~np.isnan(observed) & ~np.isnan(predicted)
    if not any(mask):
        return {'Corr²': np.nan, 'MAE': np.nan, 'Percent Bias': np.nan}
    
    obs_clean = observed[mask]
    pred_clean = predicted[mask]
    
    if len(obs_clean) < 2:
        return {'Corr²': np.nan, 'MAE': np.nan, 'Percent Bias': np.nan}
    
    # Calculate correlation squared instead of R-squared
    corr, _ = pearsonr(obs_clean, pred_clean)
    corr_squared = corr ** 2
    
    # Calculate MAE and percent bias
    mae = np.mean(np.abs(obs_clean - pred_clean))
    pbias = calculate_percent_bias(obs_clean, pred_clean)
    
    return {'Corr²': round(corr_squared, 3), 'MAE': round(mae, 3), 'Percent Bias': round(pbias, 2)}

# Define function to load data files
def load_csv_file(file_path):
    """Load a CSV file and handle errors"""
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        # Create empty dataframe with expected columns
        return pd.DataFrame({'Year': [], **{m: [] for m in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']}})

# Define the months for all data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
wyt_types = ['W', 'AN', 'BN', 'D', 'C']  # Water Year Types

# Load all data files from the Results folder
try:
    # Try to load all the required data files from the Results folder
    historical_flows = load_csv_file('Results/historical_flows.csv')
    customLSTM_flows = load_csv_file('Results/forecasted_flows_CustomLSTM.csv')
    vanillaLSTM_flows = load_csv_file('Results/forecasted_flows_vanilaLSTM.csv')
    MSO_flows = load_csv_file('Results/MSO_forecast_9.csv')
    B120_flows = load_csv_file('Results/B120_forecast_9.csv')
    wyt_data = load_csv_file('Results/WYT.csv')
    
    # Get all available years from the data
    all_years = sorted(set(pd.concat([
        historical_flows['Year'], 
        customLSTM_flows['Year'],
        vanillaLSTM_flows['Year'], 
        MSO_flows['Year'],
        B120_flows['Year']
    ]).dropna().astype(int).tolist()))
    
    # Define training and test periods - use data from files if possible
    min_year = min(all_years) if all_years else 1922
    max_year = max(all_years) if all_years else 2021
    
    print(f"Data loaded successfully! Years range from {min_year} to {max_year}")

except Exception as e:
    print(f"Failed to load data files from Results folder: {e}")
    print("Using sample data instead...")
    
    # Generate sample data for demonstration
    years = list(range(1922, 2022))
    
    # Create sample data generation function
    def create_sample_data(years, months, base_value=100, noise_level=30, missing_prob=0.05):
        data = {'Year': years}
        
        for month in months:
            # Create seasonal pattern with some randomness
            month_idx = months.index(month)
            seasonal_factor = 1 + 0.5 * np.sin(month_idx / len(months) * 2 * np.pi)
            
            # Generate values with randomness and some missing data
            values = []
            for year in years:
                if np.random.random() < missing_prob:
                    values.append(np.nan)  # Some missing data
                else:
                    # Add some long-term trend and yearly variation
                    year_factor = 1 + 0.002 * (year - 1922)
                    value = base_value * seasonal_factor * year_factor * (1 + np.random.normal(0, noise_level/100))
                    values.append(round(value, 1))
            
            data[month] = values
        
        return pd.DataFrame(data)
    
    # Create sample data for each model
    historical_flows = create_sample_data(years, months, base_value=100, noise_level=20)
    customLSTM_flows = create_sample_data(years, months, base_value=95, noise_level=25)
    vanillaLSTM_flows = create_sample_data(years, months, base_value=98, noise_level=30)
    MSO_flows = create_sample_data(years, months, base_value=105, noise_level=35)
    B120_flows = create_sample_data(years, months, base_value=102, noise_level=28)
    
    # Create WYT data
    wyt_data = pd.DataFrame({
        'Year': years,
        'WYT': np.random.choice(wyt_types, size=len(years))
    })
    
    all_years = years

# Define training and test periods based on your specification
training_years = list(range(1923, 1970)) + list(range(1991, 2023))
test_years = list(range(1970, 1991))

# Calculate total accumulated flow for each dataset (sum of Jan-Sep)
def calculate_total_accumulated_flow(df):
    """Calculate the total accumulated flow (Jan-Sep) for each year"""
    result = df.copy()
    result['Total_Flow'] = result[months].sum(axis=1)
    return result

# Calculate accumulated totals for each dataset
historical_flows_total = calculate_total_accumulated_flow(historical_flows)
customLSTM_flows_total = calculate_total_accumulated_flow(customLSTM_flows)
vanillaLSTM_flows_total = calculate_total_accumulated_flow(vanillaLSTM_flows)
MSO_flows_total = calculate_total_accumulated_flow(MSO_flows)
B120_flows_total = calculate_total_accumulated_flow(B120_flows)

# Create lookup dictionaries for data access
data_dict = {
    'Observed': historical_flows,
    'Customized LSTM': customLSTM_flows,  # Changed from 'CustomLSTM' to 'Customized LSTM'
    'VanillaLSTM': vanillaLSTM_flows,
    'Benchmark': MSO_flows,  # Changed from 'MSO' to 'Benchmark'
    'B120': B120_flows
}

data_dict_total = {
    'Observed': historical_flows_total,
    'Customized LSTM': customLSTM_flows_total,  # Changed from 'CustomLSTM' to 'Customized LSTM'
    'VanillaLSTM': vanillaLSTM_flows_total,
    'Benchmark': MSO_flows_total,  # Changed from 'MSO' to 'Benchmark'
    'B120': B120_flows_total
}

# Define model colors and markers for consistent visualization
model_colors = {
    'Observed': 'black',
    'Customized LSTM': 'blue',  # Changed from 'CustomLSTM' to 'Customized LSTM'
    'VanillaLSTM': 'green',
    'Benchmark': 'red',  # Changed from 'MSO' to 'Benchmark'
    'B120': 'purple'
}

# Define line styles (Bokeh dash patterns)
model_line_styles = {
    'Observed': 'solid',
    'Customized LSTM': 'dashed',  # Changed from 'CustomLSTM' to 'Customized LSTM'
    'VanillaLSTM': 'dotted',
    'Benchmark': 'dotdash',  # Changed from 'MSO' to 'Benchmark'
    'B120': 'dashdot'
}

# Create Panel widgets for user interaction
# Monthly tab period selection
monthly_period_select = pn.widgets.RadioButtonGroup(
    name='Period', 
    options=['Training', 'Test', 'All'],
    button_type='primary'
)

monthly_year_range_slider = pn.widgets.RangeSlider(
    name='Monthly Year Range',
    start=1922,
    end=2021,
    value=(1970, 1990),
    step=1
)

# Accumulated tab period selection (separate)
accum_period_select = pn.widgets.RadioButtonGroup(
    name='Period', 
    options=['Training', 'Test', 'All'],
    button_type='primary'
)

accum_year_range_slider = pn.widgets.RangeSlider(
    name='Accumulated Year Range',
    start=1922,
    end=2021,
    value=(1970, 1990),
    step=1
)

# Common widgets
year_select = pn.widgets.MultiChoice(
    name='Select Specific Years',
    options=[str(y) for y in all_years],
    value=[]
)

wyt_select = pn.widgets.MultiChoice(
    name='Water Year Type',
    options=wyt_types,
    value=[]
)

# Model selection with checkboxes
model_select = pn.widgets.CheckBoxGroup(
    name='Models',
    options=['Observed', 'Customized LSTM', 'VanillaLSTM', 'Benchmark', 'B120'],  # Changed model names here
    value=['Observed', 'Customized LSTM']  # Updated default value
)

# Functions to filter years based on selections (separate for monthly and accumulated)
def filter_monthly_years():
    filtered_years = []
    
    # Filter by period
    if monthly_period_select.value == 'Training':
        filtered_years = training_years
    elif monthly_period_select.value == 'Test':
        filtered_years = test_years
    else:  # All
        filtered_years = list(range(monthly_year_range_slider.value[0], monthly_year_range_slider.value[1] + 1))
    
    # Filter by selected years (if any)
    if year_select.value:
        selected_years = [int(y) for y in year_select.value]
        filtered_years = [y for y in filtered_years if y in selected_years]
    
    # Filter by WYT (if any)
    if wyt_select.value:
        wyt_years = wyt_data[wyt_data['WYT'].isin(wyt_select.value)]['Year'].tolist()
        filtered_years = [y for y in filtered_years if y in wyt_years]
    
    return filtered_years

def filter_accum_years():
    filtered_years = []
    
    # Filter by period
    if accum_period_select.value == 'Training':
        filtered_years = training_years
    elif accum_period_select.value == 'Test':
        filtered_years = test_years
    else:  # All
        filtered_years = list(range(accum_year_range_slider.value[0], accum_year_range_slider.value[1] + 1))
    
    # Filter by selected years (if any)
    if year_select.value:
        selected_years = [int(y) for y in year_select.value]
        filtered_years = [y for y in filtered_years if y in selected_years]
    
    # Filter by WYT (if any)
    if wyt_select.value:
        wyt_years = wyt_data[wyt_data['WYT'].isin(wyt_select.value)]['Year'].tolist()
        filtered_years = [y for y in filtered_years if y in wyt_years]
    
    return filtered_years

# Function to create monthly evaluation plot for a specific year
def create_monthly_eval_plot(year):
    # Get WYT for this year
    wyt = wyt_data[wyt_data['Year'] == year]['WYT'].values[0] if year in wyt_data['Year'].values else 'Unknown'
    
    # Create figure
    p = figure(
        title=f"Monthly Flow Evaluation - Year: {year} (WYT: {wyt})",
        x_range=months,  # Use months as categorical x-range
        width=800,
        height=400,
        toolbar_location="right"
    )
    
    # Legend items
    legend_items = []
    
    # Add data for each selected model
    for model in model_select.value:
        df = data_dict[model]
        
        # Check if this model has data for this year
        if year not in df['Year'].values:
            continue
            
        row = df[df['Year'] == year].iloc[0]
        
        # Check for missing data in this year
        has_missing = any(pd.isna(row[m]) for m in months)
        if has_missing:
            # Create a clean list of months and values without NaN
            valid_months = []
            valid_values = []
            for m in months:
                if not pd.isna(row[m]):
                    valid_months.append(m)
                    valid_values.append(row[m])
                    
            if not valid_months:
                continue  # Skip if no valid data
                
            # Create data source with only valid data
            source = ColumnDataSource(data=dict(
                x=valid_months,
                y=valid_values,
                model=[model] * len(valid_months)
            ))
        else:
            # All data is valid
            values = [row[m] for m in months]
            source = ColumnDataSource(data=dict(
                x=months,
                y=values,
                model=[model] * len(months)
            ))
        
        # Line glyph
        line = p.line(
            'x', 'y', source=source, 
            line_color=model_colors[model], 
            line_width=2,
            line_dash=model_line_styles[model]
        )
        
        # Use scatter instead of circle (which is deprecated with size parameter)
        scatter = p.scatter(
            'x', 'y', source=source,
            size=8,
            color=model_colors[model],
            fill_color=model_colors[model]
        )
        
        # Add to legend
        legend_items.append((model, [line, scatter]))
            
    # Add hover tool
    hover = HoverTool(
        tooltips=[
            ("Model", "@model"),
            ("Month", "@x"),
            ("Flow", "@y{0.00}")
        ]
    )
    p.add_tools(hover)
    
    # Add legend
    legend = Legend(items=legend_items, location="top_center")
    legend.click_policy = "hide"
    p.add_layout(legend, 'above')
    
    # Axis labels
    p.xaxis.axis_label = "Month"
    p.yaxis.axis_label = "Flow"
    
    # Calculate metrics if Observed is selected
    metrics_div = None
    if 'Observed' in model_select.value:
        observed_df = data_dict['Observed']
        if year in observed_df['Year'].values:
            observed_row = observed_df[observed_df['Year'] == year].iloc[0]
            
            metrics_html = "<div style='padding: 10px; border: 1px solid #ddd; background-color: #f9f9f9;'>"
            metrics_html += f"<h3>Performance Metrics</h3>"
            
            for model in model_select.value:
                if model != 'Observed' and year in data_dict[model]['Year'].values:
                    model_row = data_dict[model][data_dict[model]['Year'] == year].iloc[0]
                    
                    # Find common valid months between observed and model
                    valid_months = []
                    observed_values = []
                    model_values = []
                    
                    for m in months:
                        if not pd.isna(observed_row[m]) and not pd.isna(model_row[m]):
                            valid_months.append(m)
                            observed_values.append(observed_row[m])
                            model_values.append(model_row[m])
                    
                    if valid_months:  # Only calculate metrics if we have common valid data
                        metrics = calculate_metrics(np.array(observed_values), np.array(model_values))
                        
                        metrics_html += f"<h4>{model}</h4>"
                        metrics_html += f"<p>Corr² = {metrics['Corr²']}<br>"
                        metrics_html += f"MAE = {metrics['MAE']}<br>"
                        metrics_html += f"Bias = {metrics['Percent Bias']}%</p>"
                    else:
                        metrics_html += f"<h4>{model}</h4><p>No valid data to calculate metrics</p>"
            
            metrics_html += "</div>"
            metrics_div = pn.pane.HTML(metrics_html, width=300)
    
    # Return plot and metrics as a row
    if metrics_div:
        return pn.Row(p, metrics_div)
    else:
        return p

# Function to create accumulated evaluation plot with NO parameters
def create_total_eval_plot():
    """Create a single plot for accumulated flow comparison across selected years"""
    # Filter years based on user selections
    years_to_display = filter_accum_years()  # Using accumulated period filter
    
    if not years_to_display or not model_select.value:
        return pn.pane.Markdown("No data to display. Please adjust your selections.")
    
    # For each model, get years with valid data
    all_valid_years = set()
    model_data = {}
    
    for model in model_select.value:
        df = data_dict_total[model]
        
        # Filter for selected years that have data in this model
        valid_years = []
        valid_flows = []
        valid_wyt = []
        
        for year in years_to_display:
            year_data = df[df['Year'] == year]
            if not year_data.empty and not pd.isna(year_data['Total_Flow'].values[0]):
                valid_years.append(year)
                valid_flows.append(year_data['Total_Flow'].values[0])
                
                # Get WYT
                wyt = wyt_data[wyt_data['Year'] == year]['WYT'].values[0] if year in wyt_data['Year'].values else 'Unknown'
                valid_wyt.append(wyt)
                
                all_valid_years.add(year)
        
        if valid_years:  # Only store if we have valid years
            model_data[model] = {
                'years': valid_years,
                'flows': valid_flows,
                'wyt': valid_wyt
            }
    
    # Get sorted years for x-axis
    if not all_valid_years:
        return pn.pane.Markdown("No valid data to display. Please adjust your selections.")
        
    sorted_years = sorted(list(all_valid_years))
    year_strs = [str(y) for y in sorted_years]
    
    # Create figure with categorical x-range
    p = figure(
        title="Annual Accumulated Flow Evaluation (Jan-Sep Total)",
        x_range=FactorRange(factors=year_strs),  # Use FactorRange instead of setting factors attribute
        width=900,
        height=500,
        toolbar_location="right"
    )
    
    # Legend items
    legend_items = []
    
    # Add data series for each model
    for model, data in model_data.items():
        if not data['years']:
            continue
            
        # Create data source
        source = ColumnDataSource(data=dict(
            x=[str(y) for y in data['years']],
            y=data['flows'],
            model=[model] * len(data['years']),
            year=data['years'],
            wyt=data['wyt']
        ))
        
        # Add the line (only if we have multiple points)
        if len(data['years']) > 1:
            line = p.line(
                'x', 'y', source=source, 
                line_color=model_colors[model], 
                line_width=2,
                line_dash=model_line_styles[model]
            )
            
            # Add to legend with the line
            legend_items.append((model, [line]))
        
        # Add markers (using scatter instead of circle to avoid deprecation warning)
        scatter = p.scatter(
            'x', 'y', source=source,
            size=8,
            color=model_colors[model],
            fill_color=model_colors[model]
        )
        
        # If we didn't add a line, add to legend with just the marker
        if len(data['years']) <= 1:
            legend_items.append((model, [scatter]))
    
    # Add hover tool with additional info
    hover = HoverTool(
        tooltips=[
            ("Year", "@year"),
            ("WYT", "@wyt"),
            ("Model", "@model"),
            ("Total Flow", "@y{0.00}")
        ]
    )
    p.add_tools(hover)
    
    # Add legend
    if legend_items:
        legend = Legend(items=legend_items, location="top_right")
        legend.click_policy = "hide"
        p.add_layout(legend)
    
    # Axis labels and styling
    p.xaxis.axis_label = "Year"
    p.yaxis.axis_label = "Total Flow (Jan-Sep)"
    p.xaxis.major_label_orientation = 45  # Rotate year labels for better visibility
    
    # Calculate metrics if Observed is selected
    metrics_div = None
    if 'Observed' in model_select.value and 'Observed' in model_data:
        observed_years = set(model_data['Observed']['years'])
        
        metrics_html = "<div style='padding: 10px; border: 1px solid #ddd; background-color: #f9f9f9;'>"
        metrics_html += f"<h3>Overall Performance Metrics</h3>"
        
        for model in model_select.value:
            if model != 'Observed' and model in model_data:
                model_years = set(model_data[model]['years'])
                
                # Get common years between observed and model
                common_years = observed_years.intersection(model_years)
                
                if common_years:
                    # Get data for common years
                    obs_data = []
                    model_data_values = []
                    
                    for year in sorted(common_years):
                        # Find indices in both datasets
                        obs_idx = model_data['Observed']['years'].index(year)
                        model_idx = model_data[model]['years'].index(year)
                        
                        obs_data.append(model_data['Observed']['flows'][obs_idx])
                        model_data_values.append(model_data[model]['flows'][model_idx])
                    
                    # Calculate metrics using correlation squared
                    metrics = calculate_metrics(np.array(obs_data), np.array(model_data_values))
                    
                    metrics_html += f"<h4>{model}</h4>"
                    metrics_html += f"<p>Years: {len(common_years)}<br>"
                    metrics_html += f"Corr² = {metrics['Corr²']}<br>"
                    metrics_html += f"MAE = {metrics['MAE']}<br>"
                    metrics_html += f"Bias = {metrics['Percent Bias']}%</p>"
                else:
                    metrics_html += f"<h4>{model}</h4><p>No common years with observed data</p>"
        
        metrics_html += "</div>"
        metrics_div = pn.pane.HTML(metrics_html, width=300)
    
    # Return the plot and metrics
    if metrics_div:
        return pn.Row(p, metrics_div)
    else:
        return p

# Function to update available years based on period selection (monthly)
def update_monthly_year_options(event):
    if event.new == 'Training':
        monthly_year_range_slider.value = (1923, 2022)
    elif event.new == 'Test':
        monthly_year_range_slider.value = (1970, 1990)
    else:  # All
        monthly_year_range_slider.value = (min_year, max_year)

monthly_period_select.param.watch(update_monthly_year_options, 'value')

# Function to update available years based on period selection (accumulated)
def update_accum_year_options(event):
    if event.new == 'Training':
        accum_year_range_slider.value = (1923, 2022)
    elif event.new == 'Test':
        accum_year_range_slider.value = (1970, 1990)
    else:  # All
        accum_year_range_slider.value = (min_year, max_year)

accum_period_select.param.watch(update_accum_year_options, 'value')

# Create dashboard callback for monthly tab
@pn.depends(monthly_period_select.param.value, monthly_year_range_slider.param.value, 
            year_select.param.value, wyt_select.param.value, model_select.param.value)
def update_monthly_dashboard(period, year_range, selected_years, selected_wyt, selected_models):
    # Filter years based on criteria
    years_to_display = filter_monthly_years()
    
    if not years_to_display:
        return pn.pane.Markdown("## No years match the selected criteria. Please adjust your filters.")
    
    if not selected_models:
        return pn.pane.Markdown("## Please select at least one model to display.")

    # Create plots for each year (monthly evaluation)
    monthly_plots = []
    for year in years_to_display:
        monthly_plots.append(create_monthly_eval_plot(year))
    
    # Return monthly plots 
    return pn.Column(*monthly_plots, sizing_mode='stretch_width')

# Create dashboard callback for accumulated tab
@pn.depends(accum_period_select.param.value, accum_year_range_slider.param.value, 
            year_select.param.value, wyt_select.param.value, model_select.param.value)
def update_accum_dashboard(period, year_range, selected_years, selected_wyt, selected_models):
    # Filter years based on criteria
    years_to_display = filter_accum_years()
    
    if not years_to_display:
        return pn.pane.Markdown("## No years match the selected criteria. Please adjust your filters.")
    
    if not selected_models:
        return pn.pane.Markdown("## Please select at least one model to display.")

    # Create accumulated evaluation (a single plot for all selected years)
    accumulated_plot = create_total_eval_plot()
    
    # Return accumulated plot
    return pn.Column(accumulated_plot, sizing_mode='stretch_width')

# Layout
dashboard_title = pn.pane.Markdown("# Flow Forecast Model Evaluation Dashboard")

# Common controls
model_controls = pn.Column(
    pn.pane.Markdown("### Select Models"),
    model_select,
    width=250
)

filter_controls = pn.Column(
    pn.pane.Markdown("### Additional Filters"),
    year_select,
    wyt_select,
    width=250
)

# Monthly tab controls
monthly_controls = pn.Column(
    pn.pane.Markdown("### Monthly Period Selection"),
    monthly_period_select,
    monthly_year_range_slider,
    width=300
)

# Accumulated tab controls
accum_controls = pn.Column(
    pn.pane.Markdown("### Accumulated Period Selection"),
    accum_period_select,
    accum_year_range_slider,
    width=300
)

# Create layout for each tab
monthly_tab = pn.Column(
    pn.Row(monthly_controls, model_controls, filter_controls),
    update_monthly_dashboard
)

accum_tab = pn.Column(
    pn.Row(accum_controls, model_controls, filter_controls),
    update_accum_dashboard
)

# Create tabs for monthly and accumulated evaluations
tabs = pn.Tabs(
    ('Monthly Evaluation', monthly_tab),
    ('Accumulated Evaluation', accum_tab)
)

# Main layout
main_layout = pn.Column(
    dashboard_title,
    tabs,
    sizing_mode='stretch_width'
)

# Serve dashboard
dashboard = pn.template.FastListTemplate(
    title="Flow Forecast Model Evaluation",
    main=[main_layout],
    header_background="#1f77b4",
)

dashboard.servable()

Data loaded successfully! Years range from 1922 to 2021
Launching server at http://localhost:64537
